In [6]:
import cudaq
from cudaq import spin

from typing import List

import collections

import time
from datetime import datetime

In [2]:
class Graph():
    '''
    A graph is saved in both an adjoint matrix and edge list.
    '''
    def __init__(self, v:list=None, edges:list=None,adjoint=None) -> None:

        self.v = v
        self.n_v = len(v)
        self.e = edges
        self.adj = adjoint

        if self.adj is None:
            self._edges_to_adjoint()

        if self.e is None:
            self._adjoint_to_edges()

        self.v2i = {v[i]:i for i in range(self.n_v)}

    def _edges_to_adjoint(self) -> None:
        self.adj = np.zeros((self.n_v, self.n_v))
        for edge in self.e:
            v1 = edge[0]
            v2 = edge[1]
            if len(edge) < 3:
                w = 1
            else:
                w = edge[2]
            self.adj[v1][v2] = w
            self.adj[v2][v1] = w

    def _adjoint_to_edges(self) -> None:
        self.e = []

        for i in range(self.n_v):
            for j in range(i+1, self.n_v):
                if self.adj[i][j] != 0:
                    self.e.append((i, j, self.adj[i][j].item()))

    def graph_partition(self, n:int, policy:str='random',n_sub=1) -> list:
        '''
        n : Allowable qubit number.

        policy : Partition strategy. Default is 'random'. Another is 'modularity', which partitions graph basing on greedy modularity method.

        n_sub : number of subgraphs. Only use in 'modularity'.
        ''' 
        H = []
        v = self.v

        if policy == 'modularity':
            G = nx.Graph()
            G.add_nodes_from(v)
            for x in self.e:
                G.add_edge(x[0],x[1],weight=x[2])
            c = greedy_modularity_communities(G,n_communities=n_sub)
            sub_list = [list(x) for x in c]
            for x in sub_list:
                if len(x) > n:
                    n_ssub = math.ceil(len(x) / n)
                    
                    ssub_list = [x[n*i:n*(i+1)] for i in range(n_ssub)]
                    for i in range(n_ssub):
                        A = self.adj[ssub_list[i]][:,ssub_list[i]]
                        H.append(Graph(v=ssub_list[i], adjoint=A))
                else:
                    A = self.adj[x][:,x]
                    H.append(Graph(v=x, adjoint=A))
        if policy == 'random':
            n_sub = math.ceil(self.n_v / n)
            np.random.shuffle(v)
            sub_list = [v[n*i:n*(i+1)] for i in range(n_sub)]
            for i in range(n_sub):
                A = self.adj[sub_list[i]][:,sub_list[i]]
                H.append(Graph(v=sub_list[i], adjoint=A))
        return H

In [3]:
def get_hamiltonian(edges):
    """
    Get the Hamiltonian mapping for an arbitrary graph

    Parameters
    ----------
    edges : List[Tuple[int, int]]
        List of edges in the graph
    
    Returns
    -------
    hamiltonian : cudaq.Operator
        Hamiltonian operator
    qubit_count : int
        Number of qubits required to represent the graph
    """
    # avoid 0 term in the Hamiltonian
    hamiltonian = 0.5 * spin.z(edges[0][0]) * spin.z(edges[0][1])
    for u, v in edges[1:]:
        hamiltonian += 0.5 * spin.z(u) * spin.z(v)
    return hamiltonian

def qaoa(G:Graph, shots:int=1000, n_layers:int=1, const=0, save_file=False):
    '''
    standard qaoa for max cut
    --------------------------
    G : Graph 

    shots : number of circuit shots to obtain optimal bitstring

    n_layers : number of QAOA layers

    const : constant in max cut objective function


    Return cut value and solution
    '''
    qubit_count = G.n_v
    edges = G.e
    # subgraph with no edges, any partition is optimal
    if edges == []:
        return const, format(0,"0{}b".format(qubit_count))[::-1]

    now = datetime.now()
    if save_file:
        filename = now.strftime('data/%Y%m%d-%H%M%S-') + f'Q{qubit_count}L{n_layers}N{shots}' + ".csv"
        fp = open(filename, "w")

    @cudaq.kernel
    def kernel_qaoa(qubit_count: int, layer_count: int, thetas: List[float]):
        """QAOA ansatz for Max-Cut"""
        qvector = cudaq.qvector(qubit_count)
    
        # Create superposition
        h(qvector)
    
        # Loop over the layers
        for layer in range(layer_count):
            # Loop over the qubits
            # Problem unitary
            for qubit in range(qubit_count):
                x.ctrl(qvector[qubit], qvector[(qubit + 1) % qubit_count])
                rz(2.0 * thetas[layer], qvector[(qubit + 1) % qubit_count])
                x.ctrl(qvector[qubit], qvector[(qubit + 1) % qubit_count])
    
            # Mixer unitary
            for qubit in range(qubit_count):
                rx(2.0 * thetas[layer + layer_count], qvector[qubit])
    
    
    # Specify the optimizer and its initial parameters. Make it repeatable.
    cudaq.set_random_seed(13)
    optimizer = cudaq.optimizers.COBYLA()
    np.random.seed(13)
    parameter_count = 2 * n_layers
    optimizer.initial_parameters = np.random.uniform(-np.pi / 8.0, np.pi / 8.0,
                                                     parameter_count)   

    hamiltonian = get_hamiltonian(G.e) 
    
    # Define the objective, return `<state(params) | H | state(params)>`
    def objective(parameters):
        result = cudaq.observe(kernel_qaoa, hamiltonian, qubit_count, n_layers, parameters).expectation()
        if save_file:
            fp.write(str(time.time()) + "," + str(result) + "\n")
        return result
    
    
    # Optimize!
    optimal_expectation, optimal_parameters = optimizer.optimize(dimensions=parameter_count, function=objective)
        
    if save_file:
        fp.close()
    
    # Sample the circuit using the optimized parameters
    counts = cudaq.sample(kernel_qaoa, qubit_count, n_layers, optimal_parameters, shots_count=shots)
    end = time.time()
    sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    sol = sorted_counts[0][0]

    obj = 0
    for edge in edges:
        obj += 0.5 * (2*(sol[edge[0]]==sol[edge[1]]) - 1)
    return const - obj, sol


In [5]:
hex_graph = [(0,1), (0,2), (0,3), (0,4), (1,2), (1,4), (2,3), (2,4), (2,5), (3,5), (4,5)]
G = Graph(v=list(range(6)), edges=hex_graph)

ham = get_hamiltonian(G.e)
print(ham)
cutval, sol = qaoa(G, n_layers=2)

print(cutval, sol)

[0.5+0j] IIZIIZ
[0.5+0j] ZIIZII
[0.5+0j] ZZIIII
[0.5+0j] IIIZIZ
[0.5+0j] ZIIIZI
[0.5+0j] IIIIZZ
[0.5+0j] IIZIZI
[0.5+0j] IZZIII
[0.5+0j] IIZZII
[0.5+0j] IZIIZI
[0.5+0j] ZIZIII

1.5 010101
